In [ ]:
from mesa import Agent, Model
from mesa.space import SingleGrid
from mesa.time import RandomActivation, SimultaneousActivation
from mesa.datacollection import DataCollector
import numpy as np
import matplotlib.pyplot as plt
import random as rm

from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer


In [ ]:
class KeynesAgent(Agent):
    """
         Create a new PostKeynesian agent.
         Args:
            model: class model in wich the agent lives
            pos: Posicion of the agent (tuple or list?)
            v_c: Consume vector: informacion about the consuming behavior of the agent
                    (tuple, list, or numpy vector??)
        """
    D=dict(B=np.array([1,0,0]), M=np.array([0,1,0]), A=np.array([0,0,1]),
           MB=np.array([1,1,0]), AB=np.array([1,0,1]), AM=np.array([0,1,1]),
           AMB=np.array([1,1,1]))
    
    # Clases sociales 
    
    D_med=[D['M'], D['MB'], D['AM']] # media: reglas 2,3,4,5
    D_alt=[D['A'], D['AB'], D['AMB']] # alta: reglas 2,3,4,5

  

    def __init__(self, model, pos, v_c):
        #self.model=model
        super().__init__(pos, model)
        self.pos = pos
        self.v_c = v_c
        

    def social_class(self):
          #boolean selectors
        bool_bajo=(self.v_c == KeynesAgent.D['B']).all()
        bool_med = [(self.v_c == j).all() for j in KeynesAgent.D_med]
        bool_alt = [(self.v_c == j).all() for j in KeynesAgent.D_alt]
        #print(KeynesAgent.D_med)
        if bool_bajo:
        
            print('Es de clase baja')
            #agent.v_c = agent.v_c
        
        elif True in bool_med:
        
            print('Es de clase media')
            #med_rules(agent)   
        
        else:
            print('Es de clase alta')
            #alt_rules(agent) 
        
    #Metodos a escribir
    def step(self):
        ind = rm.choice(range(3))
        self.v_c[ind] = abs(self.v_c[ind]-1)
        #pass
        

In [ ]:
class KeynesModel(Model):
    
    """Model class for the Postkeynessian consumer model"""
    def __init__(self, N, m, n, n_b, n_m, n_a):
        """
        N = numero de agentes
        m x n = numero total de celdas del modelo
            m = numero de filas
            n = numero de columnas
        *args = lista/tupla (??) con entradas n_b , n_m , n_a, donde
            n_b = numero de agentes de clase baja
            n_m = numero de agentes de clase media
            n_a = numero de agentes de clase alta
        """
        args=(n_b,n_m,n_a)
        if np.array(args).sum() != N:
            print("Error, la suma de agentes de cada clase no es igual a N")
        else:
            self.num_agents = N
            self.grid = SingleGrid(m, n, torus=True)
            self.schedule = RandomActivation(self)
            self.running = True
        
            #CREATE AGENTS
            # matrix of all coordinates, ramdom choosing from this to set agent's position and 
            # then  must asign V_c from a list of numpy arrays
            # order: clase baja, clase media, clase alta
            
            M=[(i,j) for i in range(m) for j in range(n)] #matriz de posiciones
            n_b, n_m, n_a=args[0], args[1], args[2] #linea innsecesaria
            for j in range(3):
                v_c=[0,0,0]
                for i in range(args[j]):
                    v_c[j]=1
                    #x = self.random.randrange(m)
                    #y = self.random.randrange(n)
                    #pos=(x,y)
                    pos=M.pop(M.index(rm.choice(M)))
                    #print(pos)
                    v_c=np.array(v_c)
                    a=KeynesAgent(self,pos,v_c)
                    self.schedule.add(a)
                    self.grid.position_agent(a,pos) 
            

    def step(self):
        #self.datacollector.collect(self)
        self.schedule.step()
        

In [ ]:
model=KeynesModel(100,10,10,50,25,25)
#model.grid.get_neighbors((1,1),moore=True)# vecinos

In [ ]:
a=model.schedule.agents[98]
print(a.v_c)

In [ ]:
a.social_class()

In [ ]:
#probando un step
model.grid.get_neighbors((1,1),moore=True) #así obtengo a los vecinos

In [ ]:
#comprobando la suma de los vectores de consumo
vecs=[a.v_c for a in model.schedule.agents]
vecs=np.array(vecs)
vecs.sum(0)

In [ ]:
a=model.schedule.agents[0].v_c

In [ ]:
(a==a).all()

In [ ]:
False in model.schedule.agents[0].v_c == (1,0,0)

# Visualización inicial

In [ ]:
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer

In [ ]:
def agent_portrayal(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "Layer": 0,
                 "r": 0.5}
    
    D=dict(B=np.array([1,0,0]), M=np.array([0,1,0]), A=np.array([0,0,1]),
       MB=np.array([1,1,0]), AB=np.array([1,0,1]), AM=np.array([0,1,1]),
       AMB=np.array([1,1,1]))
    
    if (agent.v_c == D['B']).all() :
        portrayal["Color"] = "red"
        
    elif (agent.v_c == D['M']).all():
        portrayal["Color"] = "green" 
#     else:
#         portrayal["Color"] = "blue"
    elif (agent.v_c == D['A']).all():
        portrayal["Color"] = "blue"
        
    elif (agent.v_c == D['MB']).all():
        portrayal["Color"] = "yellow"
        
    elif (agent.v_c == D['AB']).all():
        portrayal["Color"] = "#FF1493" #rosa mexicano
        
    elif (agent.v_c == D['AM']).all():
        portrayal["Color"] = "#48D1CC" #turquesa bonito
        
    elif (agent.v_c == D['AMB']).all():
        portrayal["Color"] = "#000000" #negro
    else :
        portrayal["Color"] = "#FFFFFF" #blanco
        
        
    return portrayal

In [ ]:
grid = CanvasGrid(agent_portrayal, 10, 10, 500, 500)

In [ ]:
grid = CanvasGrid(agent_portrayal, 10, 10, 500, 500)
server = ModularServer(KeynesModel,
                       [grid],
                       "Post-Keynesian Model",
                       {"N":100, "m":10, "n":10, "n_b":50, "n_m":25, "n_a":25})
server.port = 8553 # The default
server.launch()

# Testing functions

In [ ]:
def creating_agents(n,m,*args):
    M=[(i+1,j+1) for i in range(m) for j in range(n)] #matriz de posiciones
    n_b, n_m, n_a=args[0], args[1], args[2] # linea innecesaria
    if np.array(args).sum() != N:
        print('Error: La cantidad de agentes en las clases no es la correcta ')
    else:
        for i in range(args[0]):
            pos=M.pop(M.index(rm.choice(M)))
            v_c=np.array((1,0,0))
            #print(f'position asigned{pos}')
        #print('Las posisiones restantes son:')
        #print(M)
        return M 

In [ ]:
def vc_update(self):
    
    #dictionary with all possible V_c's
    
    D=dict(B=np.array([1,0,0]), M=np.array([0,1,0]), A=np.array([0,0,1]),
           MB=np.array([1,1,0]), AB=np.array([1,0,1]), AM=np.array([0,1,1]),
           AMB=np.array([1,1,1]))
    
    # Clases sociales 
    
    D_med=[D['M'], D['MB'], D['AM']] # media: reglas 2,3,4,5
    D_alt=[D['A'], D['AB'], D['AMB']] # alta: reglas 2,3,4,5

    #boolean selectors
    bool_bajo=(self.v_c == D['B']).all()
    bools_med = [(self.v_c == j).all() for j in D_med]
    bools_alt = [(self.v_c == j).all() for j in D_alt]


    if bool_bajo:
        pass
    elif True in bools_med:
        v_cs = np.array([neig.v_c for neig in model.grid.get_neighbors(self.pos,moore=True)])
        s = v_cs.sum(0)

        if s[1] > s[0] and s[1] > s[2]: #1 medio 
            self.v_c = D['M']

        elif s[0] < s[2] and s[1] < s[2]: #2 medio
            self.v_c = D['AM']

        elif s[2] < s[0] and s[1] < s[0]: #3 medio
            self.v_c = D['MB']

        elif s[0] == s[2]: #4 medio 
            self.v_c = D['AMB']
    else:  
        v_cs = np.array([neig.v_c for neig in model.grid.get_neighbors(self.pos,moore=True)])
        #print(f'Los vectores de consumo de los vecnios son{v_cs}')
        s = v_cs.sum(0)
        #print(f'la suma de las practicas de consumo es {s}')
        if s[2] > s[0] and s[2] > s[1]: #5 alto
            self.v_c = D['A']

        elif s[1] > s[0] and s[1] > s[2]: #6 alto
            self.v_c = D['AM']

        elif s[1] < s[0] and s[2] < s[0]: #7 alto
            self.v_c = D['AB']

        elif s[0] == s[1]: #8 alto
            self.v_c = D['AMB']

    #print(f'La actualizacion del vector de consumo es: {a.v_c}')



In [ ]:
D=dict(B=np.array([1,0,0]), M=np.array([0,1,0]), A=np.array([0,0,1]),
        MB=np.array([1,1,0]), AB=np.array([1,0,1]), AM=np.array([0,1,1]),
        AMB=np.array([1,1,1]))
    
    # Clases sociales 
    
D_med=[D['M'], D['MB'], D['AM']] # media: reglas 2,3,4,5
D_alt=[D['A'], D['AB'], D['AMB']] # alta: reglas 6,7,8,9



In [ ]:
model=KeynesModel(100,10,10,50,25,25)
a=rm.choice(model.schedule.agents)
print(f'Posicion del agente: {a.pos}')
print(f'vector de consumo: {a.v_c}')

In [ ]:
print('A qué clase pertence?')

In [ ]:
bool_bajo=(a.v_c == D['B']).all()
bool_med = [(a.v_c == j).all() for j in D_med]
bool_alt = [(a.v_c == j).all() for j in D_alt]
if bool_bajo:
    print('Es de clase baja')
elif True in bool_med:
    print('Es de clase media')
else:
    print('Es de clase alta')

In [ ]:
s=[j.v_c for j in model.grid.get_neighbors(a.pos,moore=True)]
s=np.array(s)
s.sum(0)

In [ ]:
#deberia aplicar la regla 3 media y cambiar el V_c a (1,1,0)

In [ ]:
agent=a
v_cs = np.array([neig.v_c for neig in model.grid.get_neighbors(agent.pos,moore=True)])
s=v_cs.sum(0)

In [ ]:

def med_rules(agent):
    v_cs = np.array([neig.v_c for neig in model.grid.get_neighbors(agent.pos,moore=True)])
    s = v_cs.sum(0)
    
    if s[1] > s[0] and s[1] > s[2]: #1 medio 
        agent.v_c = D['M']
            
    elif s[0] < s[2] and s[1] < s[2]: #2 medio
        agent.v_c = D['AM']
    
    elif s[2] < s[0] and s[1] < s[0]: #3 medio
        agent.v_c = D['MB']
            
    elif s[0] == s[2]: #4 medio 
        agent.v_c = D['AMB']
            
def alt_rules(agent):
    v_cs = np.array([neig.v_c for neig in model.grid.get_neighbors(agent.pos,moore=True)])
    s = v_cs.sum(0)
        
    if s[2] > s[0] and s[2] > s[1]: #5 alto
        agent.v_c = D['A']

    elif s[1] > s[0] and s[1] > s[2]: #6 alto
        agent.v_c = D['AM']

    elif s[1] < s[0] and s[2] < s[0]: #7 alto
        agent.v_c = D['AB']

    elif s[0] == s[1]: #8 alto
        agent.v_c = D['AMB']

def vc_update(agent):
    #posibles v_c
    D=dict(B=np.array([1,0,0]), M=np.array([0,1,0]), A=np.array([0,0,1]),
        MB=np.array([1,1,0]), AB=np.array([1,0,1]), AM=np.array([0,1,1]),
        AMB=np.array([1,1,1]))
    
    # Clases sociales 
    
    D_med=[D['M'], D['MB'], D['AM']] # media: reglas 2,3,4,5
    D_alt=[D['A'], D['AB'], D['AMB']] #alta: reglas 6,7,8,9
    
    #checking class
    
    bool_bajo=(agent.v_c == D['B']).all()
    bool_med = [(agent.v_c == j).all() for j in D_med]
    bool_alt = [(agent.v_c == j).all() for j in D_alt]
    
    if bool_bajo:
        
        print('Es de clase baja')
        agent.v_c = agent.v_c
        
    elif True in bool_med:
        
        print('Es de clase media')
        med_rules(agent)   
        
    else:
        print('Es de clase alta')
        alt_rules(agent)
        

In [ ]:
a=rm.choice(model.schedule.agents)
print(a.v_c)

In [ ]:
print(a.v_c)
v_cs = np.array([neig.v_c for neig in model.grid.get_neighbors(a.pos,moore=True)])
s = v_cs.sum(0)
print(s)
vc_update(a)
print(a.v_c)

In [ ]:
model=KeynesModel(100,10,10,15,35,50)
a=rm.choice(model.schedule.agents)

In [ ]:
model.step()

In [ ]:

print(a.v_c)
v_cs = np.array([neig.v_c for neig in model.grid.get_neighbors(a.pos,moore=True)])
s = v_cs.sum(0)
print(s)
vc_update(a)
print(a.v_c)

# Reglas del modelo postkeynessiano

In [ ]:
class KeynesAgent(Agent):
    """
         Create a new PostKeynesian agent.
         Args:
            model: class model in wich the agent lives
            pos: Posicion of the agent (tuple or list?)
            v_c: Consume vector: informacion about the consuming behavior of the agent
                    (tuple, list, or numpy vector??)
        """

    def __init__(self, model, pos, v_c):
        #self.model=model
        super().__init__(pos, model)
        self.pos = pos
        self.v_c = v_c
        
    def med_rules(agent):
        v_cs = np.array([neig.v_c for neig in model.grid.get_neighbors(agent.pos,moore=True)])
        s = v_cs.sum(0)

        if s[1] > s[0] and s[1] > s[2]: #1 medio 
            agent.v_c = D['M']

        elif s[0] < s[2] and s[1] < s[2]: #2 medio
            agent.v_c = D['AM']

        elif s[2] < s[0] and s[1] < s[0]: #3 medio
            agent.v_c = D['MB']

        elif s[0] == s[2]: #4 medio 
            agent.v_c = D['AMB']    
        

    def alt_rules(agent):
        v_cs = np.array([neig.v_c for neig in model.grid.get_neighbors(agent.pos,moore=True)])
        s = v_cs.sum(0)

        if s[2] > s[0] and s[2] > s[1]: #5 alto
            agent.v_c = D['A']

        elif s[1] > s[0] and s[1] > s[2]: #6 alto
            agent.v_c = D['AM']

        elif s[1] < s[0] and s[2] < s[0]: #7 alto
            agent.v_c = D['AB']

        elif s[0] == s[1]: #8 alto
            agent.v_c = D['AMB']

    def vc_update(self):
        #posibles v_c
        D=dict(B=np.array([1,0,0]), M=np.array([0,1,0]), A=np.array([0,0,1]),
            MB=np.array([1,1,0]), AB=np.array([1,0,1]), AM=np.array([0,1,1]),
            AMB=np.array([1,1,1]))

        # Clases sociales 

        D_med=[D['M'], D['MB'], D['AM']] # media: reglas 2,3,4,5
        D_alt=[D['A'], D['AB'], D['AMB']] #alta: reglas 6,7,8,9

        #checking class

        bool_bajo=(self.v_c == D['B']).all()
        bool_med = [(self.v_c == j).all() for j in D_med]
        bool_alt = [(self.v_c == j).all() for j in D_alt]

        if bool_bajo:

            #print('Es de clase baja')
            self.v_c = self.v_c

        elif True in bool_med:

            #print('Es de clase media')
            med_rules(self)   

        else:
            #print('Es de clase alta')
            alt_rules(self)
            
    def proba_update(self): #buscar como poner un atributo de clase para el diccionario de los 
                            # vectores y selección de clase y no repetirme
        pass
                
    def step(self):
        self.vc_update()


           
            
#     #Metodos a escribir
        
        

In [ ]:
class KeynesModel(Model):
    
    """Model class for the Postkeynessian consumer model"""
    def __init__(self, N, m, n, n_b, n_m, n_a):
        """
        N = numero de agentes
        m x n = numero total de celdas del modelo
            m = numero de filas
            n = numero de columnas
        *args = lista/tupla (??) con entradas n_b , n_m , n_a, donde
            n_b = numero de agentes de clase baja
            n_m = numero de agentes de clase media
            n_a = numero de agentes de clase alta
        """
        args=(n_b,n_m,n_a)
        if np.array(args).sum() != N:
            print("Error, la suma de agentes de cada clase no es igual a N")
        else:
            self.num_agents = N
            self.grid = SingleGrid(m, n, torus=True)
            #self.schedule = SimultaneousActivation(self)
            self.schedule = RandomActivation(self)
            self.running = True
        
            #CREATE AGENTS
            # matrix of all coordinates, ramdom choosing from this to set agent's position and 
            # then  must asign V_c from a list of numpy arrays
            # order: clase baja, clase media, clase alta
            
            M=[(i,j) for i in range(m) for j in range(n)] #matriz de posiciones
            n_b, n_m, n_a=args[0], args[1], args[2] #linea innsecesaria
            for j in range(3):
                v_c=[0,0,0]
                for i in range(args[j]):
                    v_c[j]=1
                    pos=M.pop(M.index(rm.choice(M)))
                    #print(pos)
                    v_c=np.array(v_c)
                    a=KeynesAgent(self,pos,v_c)
                    self.schedule.add(a)
                    self.grid.position_agent(a, pos) 

    def step(self):
        #self.datacollector.collect(self)
        self.schedule.step()
        

In [ ]:
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer

In [ ]:
def agent_portrayal(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "Layer": 0,
                 "r": 0.5}
    
    D=dict(B=np.array([1,0,0]), M=np.array([0,1,0]), A=np.array([0,0,1]),
       MB=np.array([1,1,0]), AB=np.array([1,0,1]), AM=np.array([0,1,1]),
       AMB=np.array([1,1,1]))
    
    if (agent.v_c == D['B']).all() :
        portrayal["Color"] = "red"
        
    elif (agent.v_c == D['M']).all():
        portrayal["Color"] = "green" 
#     else:
#         portrayal["Color"] = "blue"
    elif (agent.v_c == D['A']).all():
        portrayal["Color"] = "blue"
        
    elif (agent.v_c == D['MB']).all():
        portrayal["Color"] = "yellow"
        
    elif (agent.v_c == D['AB']).all():
        portrayal["Color"] = "#FF1493" #rosa mexicano
        
    elif (agent.v_c == D['AM']).all():
        portrayal["Color"] = "#48D1CC" #turquesa bonito
        
    elif (agent.v_c == D['AMB']).all():
        portrayal["Color"] = "#000000" #negro
    else :
        portrayal["Color"] = "#FFFFFF" #blanco
        
        
    return portrayal

In [ ]:
30*30

In [ ]:
N=100
m=10
n=10
nb=25
nm=50
na=25

In [ ]:
grid = CanvasGrid(agent_portrayal, m, n, 500, 500)
server = ModularServer(KeynesModel,
                       [grid],
                       "Post-Keynesian Model",
                       {"N":N, "m":m, "n":n, "n_b":nb, "n_m":nm, "n_a":na})
server.port = 8730# The default
server.launch()

In [ ]:
N=100
m=10
n=10
nb=33
nm=33
na=34
model=KeynesModel(N,m,n,nb,nm,na)

In [ ]:
a=model.schedule.agents[40]
print(f'La posicion del agente es: {a.pos}')
print(f'El V_c es: {a.v_c}')
compara=np.array([np.array([0,0,0]) for i in range(8)])
o=model.grid.get_neighbors(a.pos,moore=True)
o_pos=[j.pos for j in o]
print(f'Las pociones delos vecinos son{len(o_pos)}:')
print(o_pos)
print('Los vectores de consumo de los vecinos son')
t=[j.v_c for j in o]
print(t)
print(f'La suma de las practicas de consumo es: {np.array(t).sum(0)}')

In [ ]:
model.step()
o=model.grid.get_neighbors(a.pos,moore=True)
o_vcs=[j.v_c for j in o]
print(f'Los nuevos vectores de consumo vecinos son {len(o_vcs)}:')
print(o_vcs)
print(f'La suma de pcraticas de consumo es: {np.array(o_vcs).sum(0)}')
print('El nuevo vector de consumo del agente es:')
print(a.v_c)
# o=model.grid.get_neighbors(a.pos,moore=True)
# o_pos=[j.pos for j in o]
# print(o_pos)

In [ ]:

o=model.grid.get_neighbors(model.schedule.agents[i].pos,moore=True)

In [ ]:
oo=[[j.v_c,j.pos] for j in o]
print(oo)

In [ ]:
# this is a flip coin with random module
#draws=[]
# for i in range(1000):
#     a=rm.choices(range(2),cum_weights=[0.25,0.75])
#     draws.append(a[0]) importante el index pues choices regresa una lista
draws=[rm.choices(range(2),cum_weights=[0.25,0.75])[0] for i in range(1000)]
print(draws.count(1))
print(draws.count(0))
plt.hist(draws)
#For a script add the following line
plt.show()

In [ ]:
plt.hist(draws)
#For a script add the following line
plt.show()